In [ ]:
# Week 2 Data Analytics Exercise — **Solution**
This notebook contains the full worked solutions using the correct Eurostat renewable energy categories from the dataset `NRG_IND_REN`.
REMINDER: You are not expected to know all the functions in this script.
The aim of this exercise is to check if you understand the logic of a script.

## 0. PLOP

## 1. Install and import packages

In [20]:
# We start by loading the packages
# If you have not installed these packages, do so now
import pandas as pd
import pandasdmx as sdmx

## 2. Fetch renewable energy dataset from Eurostat (NRG_IND_REN)
For this exercise we are going to download some data directly from the internet, via Eurostat.
We use the pandasdmx package for this purpose. 

In [21]:
# Create a request object that connects to Eurostat's SDMX API.
# "ESTAT" is the official code for Eurostat in the SDMX system.
request = sdmx.Request(_____)

# Download the dataset "NRG_IND_REN" from Eurostat.
# We also add a parameter asking for data from the year 2020.
# The result is saved in the variable 'response'.
response = request.data(_____, params={"time": ____})

# Convert the SDMX response (a special format) into a regular pandas DataFrame.
# This makes it easy to filter, pivot, and analyze the data.
data = ____.to_pandas()

# Display the first few rows of the DataFrame so you can inspect the structure.
data.head()

geo  unit  freq  nrg_bal  TIME_PERIOD
AL   PC    A     REN      2004           29.620
                          2005           31.367
                          2006           32.070
                          2007           32.657
                          2008           32.448
Name: value, dtype: float64

## 3. Filter for valid renewable energy types

In [22]:
# Create a list of the renewable energy categories we want to keep.
# These are the actual Eurostat codes for:
# - REN: total renewable energy share
# - REN_ELC: renewable share in electricity
# - REN_HEAT_CL: renewable share in heating & cooling
# - REN_TRA: renewable share in transport
energy_types = ________

# Filter the dataset so we only keep the rows where the column "nrg_bal"
# (Eurostat’s code for “energy balance category”) is one of the values
# in our list above. 
#
# Explanation of the pieces:
# - data.index.get_level_values("nrg_bal") extracts the "nrg_bal" level
#   from the multi-indexed Eurostat dataframe.
# - .isin(energy_types) checks whether each row belongs to one of the
#   categories we want.
# - data[...] keeps only the matching rows.
filtered = data[data.index.get_level_values("nrg_bal").isin(energy_types)]

# Show the first few rows of the filtered dataset.
filtered.head()

geo  unit  freq  nrg_bal  TIME_PERIOD
AL   PC    A     REN      2004           29.620
                          2005           31.367
                          2006           32.070
                          2007           32.657
                          2008           32.448
Name: value, dtype: float64

## 4. Pivot into wide format

In [ ]:
# Reset the index of the filtered dataframe.
# Each row of a dataframe has an index, it's basically the number of the row in which the data is stored
# In complex datasets we can have a multi index. 
# Eurostat data often uses a MultiIndex (e.g., nrg_bal, geo, time, unit),
# so reset_index() turns these index levels into normal columns.
reset_df = _____.reset_index()

# Show the names of all columns in the dataframe.
# This helps us check which column contains the numerical values
# (usually "value" or "OBS_VALUE"), which we need for pivoting.
_____.columns

In [ ]:
# Convert the MultiIndex into normal columns so we can use them in a pivot table
df = filtered.reset_index().pivot_table(
    
    # "geo" is the region code (e.g., NUTS2 region). 
    # This becomes the row index of the pivot table.
    index=____,

    # "nrg_bal" contains the renewable energy categories 
    # (REN, REN_ELC, REN_HEAT_CL, REN_TRA).
    # These become the columns of the pivot table.
    columns=_____,

    # "value" is the actual numeric data we want to display in the table
    # (the renewable share for each region and category).
    values=_____

# Reset the index again so "geo" becomes a normal column named "Region"
).reset_index()

# Rename the columns to more understandable names for students.
# The order matches the order created by the pivot:
# Region, REN_ELC, REN_HEAT_CL, REN, REN_TRA
df._______ = [
    "Region",                    # The region identifier (e.g., NL11, DE12, etc.)
    "Renewables_Electricity",    # Share of renewables in electricity (REN_ELC)
    "Renewables_HeatingCooling", # Share in heating & cooling (REN_HEAT_CL)
    "Renewables_Total",          # Total renewable energy share (REN)
    "Renewables_Transport"       # Share in transport (REN_TRA)
]

# Display the first few rows to check that the table looks correct
_____.head()


## 5. Compute indicators

In [ ]:
# Create a new column called "High_Renewables".
# For each row, we check whether the value in the column "Renewables_Total"
# is greater than 40.

# These True/False values are stored in the new column.
df["High_Renewables"] = df[_____] > _____
#
# This expression returns True or False
# for every row in the DataFrame.
#
# Example:
# 35 > 40 → False
# 52 > 40 → True
# 48 > 40 → True
# 22 > 40 → False
#
# Show the first few rows so we can see the result of our new column.
df.head()


## 6. Fetch CO2 emissions from Eurostat
### Request CO₂ emissions data from Eurostat using the SDMX API.
### "sdg_13_10" is the official dataset code for 
### “Greenhouse gas emissions per capita”.
### Take only the year 2020

In [ ]:
# The parameter {"time": "2020"} tells Eurostat that we only want data
# for the year 2020.
response_co2 = request.data(____, params={"___": "2020"})

# Convert the SDMX response into a regular pandas DataFrame.
# The .reset_index() call turns the MultiIndex (geo, time, unit, etc.)
# into normal columns we can easily inspect and filter.
co2 = _____.to_pandas().reset_index()

# Display the first few rows so we can understand how the dataset is structured.
co2.head()

In [ ]:
# Reset the index of the CO2 dataframe.
# Eurostat datasets often use a MultiIndex (with levels like geo, time, unit).
# reset_index() turns those index levels into normal columns.
______ = co2.reset_index()

# Show the first few rows of the resulting dataframe.
# This helps us understand:
# - what the column names are,
# - which column contains the CO2 values,
# - how the data is structured before we filter or merge it.
reset_co2.head()

In [28]:
print(co2.columns)

Index(['geo', 'unit', 'freq', 'src_crf', 'TIME_PERIOD', 'value'], dtype='object')


In [ ]:
# Keep only the rows in the CO2 dataset where the region code ("geo")
# also appears in our renewables dataframe.
# This ensures we only compare CO2 emissions for the regions
# we already have renewable energy data for.
co2 = co2[co2["geo"].isin(df[_____])]

# Select only the two columns we actually need:
# - "geo": the region identifier (e.g., NL11, DE12, etc.)
# - "value": the numeric CO2 emissions value
#
# Then rename the columns so they have clear, readable names:
# "geo"   → "Region"
# "value" → "CO2_Emissions"
co2 = co2[[____, "value"]].rename(columns={____: "Region", "value": "CO2_Emissions"})

# Show the first few rows to verify that the table now has the correct structure.
co2.head()

## 7. Merge the renewable dataset with CO2 emissions

In [ ]:
# Merge the renewables dataframe (df) with the CO2 dataframe (co2).
#
# We want to match using the  "Region" column, which both dataframes share.
#
# how= should indicate that we keep ALL rows from df (the renewables data),
# and add CO2 emissions only where a matching region exists.
# If a region in df has no matching CO2 data, it will get a NaN value.
full = df.merge(co2, on=____, how=____)

# Show the first few rows of the merged dataset so we can check
# that the merge worked correctly and both renewable and CO2 values appear.
full.head()


## 8. Export final merged dataset

## 9. Plot some data

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

# Make the charts a bit prettier
sns.set_theme(style="whitegrid")

# ---------------------------------------------------------
# 1. BAR CHART: Compare renewable energy categories by region
# ---------------------------------------------------------

# Set the figure size
plt.figure(figsize=(12, 6))

# Plot each renewable energy category for each region
df_plot = df[["Region", 
              "Renewables_Electricity", 
              "Renewables_HeatingCooling", 
              "Renewables_Total", 
              "Renewables_Transport"]]

df_plot.set_index("Region").plot(kind="bar", figsize=(12, 6))

plt.title("Renewable Energy Shares by Region", fontsize=16)
plt.ylabel("Share of Renewables (%)")
plt.xlabel("Region")
plt.legend(title="Category")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



# ---------------------------------------------------------
# 2. SCATTER PLOT: Do regions with more renewables emit less CO2?
# ---------------------------------------------------------

plt.figure(figsize=(8, 6))

# Scatter plot of total renewables vs CO₂ emissions
sns.scatterplot(
    data=full,
    x="Renewables_Total",
    y="CO2_Emissions",
    hue="High_Renewables",   # Highlight high-renewable regions
    palette={True: "green", False: "red"},
    s=100
)

plt.title("Do High-Renewables Regions Emit Less CO2?", fontsize=16)
plt.xlabel("Renewables Total Share (%)")
plt.ylabel("CO₂ Emissions (per capita)")
plt.grid(True)
plt.tight_layout()
plt.show()



# ---------------------------------------------------------
# 3. SIMPLE RANKING: Top and bottom 5 regions by renewables
# ---------------------------------------------------------

# Sort by total renewables
ranked = full.sort_values("Renewables_Total", ascending=False)

print("🔝 Top 5 Regions by Renewable Energy Share:")
display(ranked.head())

print("🔻 Bottom 5 Regions by Renewable Energy Share:")
display(ranked.tail())



# ---------------------------------------------------------
# OPTIONAL: EXPORT a file for mapping in QGIS or Datawrapper
# ---------------------------------------------------------

# Save only necessary columns to a mapping-ready CSV
mapping = full[["Region", "Renewables_Total", "CO2_Emissions"]]
mapping.to_csv("mapping_ready.csv", index=False)

print("📁 'mapping_ready.csv' saved — ready for QGIS/Datawrapper mapping!")


NameError: name 'df' is not defined

<Figure size 1200x600 with 0 Axes>